In [ ]:
# Para que las funciones se actualicen si tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

In [ ]:
# Funciones propias
from utils_limpieza import * 

In [ ]:
import pandas as pd
import json
from os.path import exists
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt

In [ ]:
# Cargar datos
path = '../data/raw/tp2_muestra_diplodatos_ventas_omega_modelo_2023.csv'
ventas = pd.read_csv(path)


## Hacemos la limpieza sugerida al principio del TP2
- [x] Eliminar las variables en `cols_dropear` (Paso 1).
- [x] Renombrar a 'Otros' las subcategorías que no tengan `MODELO` (Paso 2).

In [ ]:
print(f'El df original tiene {ventas.shape[0]} registros y {ventas.shape[1]} variables.\n')

# Paso 1
cols_dropear = ['INSCRIPCION', 'CATEGORIA', 'DESCRIPCION_CATEGORIA', 'CATEGORIA (Ajustado)', 'NOMBRE']
ventas_renamed = limpiar_basic(ventas, cols_drop=cols_dropear)
print(f'Luego del paso 1 quedan se tiran {len(cols_dropear)} variables, quedando {ventas_renamed.shape[1]} variables.\n')

# Paso 2
# ventas_renamed['SUB-CATEGORIA'].nunique() # antes del paso 2
ventas_renamed = renombrar_elementos(ventas_renamed, 
                                     columna='SUB-CATEGORIA', 
                                     fill_otros='Otros')

scu_vr = ventas_renamed['SUB-CATEGORIA'].nunique()
scu_ot = ventas_renamed['SUB-CATEGORIA'].nunique() - scu_vr + 1
print(f'Luego del paso 2 quedan {scu_vr} subcategorías únicas.')
print(f'Las {scu_ot} subcategorías únicas que no tenían ningún vendedor modelo fueron agrupadas en "Otros".')
print(f'Las {scu_vr-1} subcategorías que sí tienen vendedores modelo son:')

ventas_renamed.groupby(by=['SUB-CATEGORIA'])['MODELO'].sum().sort_values(ascending=False)[:12]

## Anonimizado + simplificación de variables y valores
- [x] Simplificamos el nombre de las variables.
- [x] Anonimizamos la variable sensible `ID` (Paso 3).
- [x] Simplificamos valores en `Deposito`.
- [x] Guardamos todos estos diccionarios.

In [ ]:
# Se modifican los nombres de las columnas

# Creamos el diccionario
column_dict = {'ID_VENDEDOR': 'ID', 
                'OMEGA': 'Omega', 
                'SUB-CATEGORIA': 'Subrubro', 
                'DESC_TRATAMIENTO_FISCAL': 'Trat_Fisc_Agg', 
                'TRATAMIENTO_FISCAL': 'Trat_Fisc', 
                'TRATAMIENTO_DIFERNCIAL': 'Trat_Dif', 
                'CM04': 'CM', 
                'AÑO': 'Año', 
                'MES': 'Mes', 
                'DEPOSITO': 'Deposito', 
                'TOTAL_VENTAS': 'Ventas', 
                'PORCENTAJE_COMISION_EMPRESA': 'Alicuota', 
                'COMISION_EMPRESA': 'Recaudacion', 
                'MODELO': 'Modelo'}

with open("../references/tp2_column_dict.json", "w") as fp:
    json.dump(column_dict , fp)

# Renombramos
with open("../references/tp2_column_dict.json") as column_dict_json:
    column_dict = json.load(column_dict_json)

ventas_renamed.rename(columns = column_dict, inplace = True)
ventas_renamed.columns

In [ ]:
# Anonimizamos y guardamos el diccionario para no perder esta info
ventas_hash, dict_id = anonimizar(ventas_renamed, 'ID')

with open("../references/tp2_ID_dict.json", "w") as fp:
    json.dump(dict_id, fp)

print('¡Diccionario generado!')

In [ ]:
# Mapeo de `Deposito` a enteros
ventas_hash, dict_dep = anonimizar(ventas_renamed, 'Deposito')

with open("../references/tp2_deposito_dict.json", "w") as fp:
    json.dump(dict_dep, fp)

print('¡Diccionario generado!')

## Variable fecha (Paso 6).
Consideramos que para el análisis temporal habrá momentos en que necesitamos tener Año y Mes por separado, pero en otras ocasiones necesitamos toda la información al mismo tiempo. Por lo tanto, creamos la variable FECHA:
    $$FECHA = Año + Mes$$

In [ ]:
ventas_hash["FECHA"] = pd.to_datetime(ventas_hash['Mes'].astype(str) + '-' + ventas_hash['Año'].astype(str), format='%m-%Y')
ventas_hash[:3]

## Eliminación efecto inflación
Las variables expresadas en pesos aumentan mes a mes por efecto de la inflación. Lo que impide comparar un valor monetario de un mes contra el del mes siguiente. A fin de eliminar este efecto, una alternativa es reexpresar los valores monetarios respecto a un mes base. Por ejemplo, el último mes de la serie.

In [ ]:
ventas_ipc = ventas_hash.copy()

In [ ]:
# Obtener el último mes
año = ventas_ipc['Año'].max()
mes = ventas_ipc[ventas_ipc['Año'] == ventas_ipc['Año'].max()]['Mes'].max()
print(f"El último mes de la serie es el {mes} de {año}")

El procedimiento es el siguiente:
    $$\text{VAR}_{mm-aaaa} \frac{\text{IPC}_{06-2022}}{\text{IPC}_{mm-aaaa}}$$

Para todo valor de cada variable (VAR) correspondiente a cierto mes "mm-aaaa" se lo divide por el Índice de Precios al Consumidor (IPC) correspondiente a ese mes "mm-aaaa" y luego se lo multiplica por el IPC correspondiente al mes "06-2022".

De esta forma, todos los valores de cada variable VAR van a quedar expresados en unidades monetarias del "06-2022".



Es necesario obtener de manera externa una serie del IPC que publica Indec.

In [ ]:
precios = "../data/external/IPC_Indec.csv"

if exists(precios):
    print('Este archivo ya existe.')
else:
    print('Este archivo no existe: ¡Vamos a crearlo!')
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_06_23.xls"
    df = pd.read_excel(url, sheet_name="Índices IPC Cobertura Nacional", header=None, usecols="B:CA", skiprows=[0, 1, 2, 3, 4, 6, 7, 8], nrows=2)
    df = df.transpose()
    df.columns = ["FECHA", "INDICE"]
    df["FECHA"] = pd.to_datetime(df["FECHA"])
    df.to_csv(precios, index=False)

In [ ]:
# Abrir dataset de precios
precios = pd.read_csv(precios)
precios.head()

Luego debemos unir ambas bases de datos (ventas y precios), de acuerdo a la variable clave `FECHA`

In [ ]:
print(ventas_ipc["FECHA"].dtype)
print(precios["FECHA"].dtype)

In [ ]:
# Hay que asegurar que la variable clave tenga el mismo tipo en los dos dataframes
precios["FECHA"] = pd.to_datetime(precios["FECHA"])

In [ ]:
print(ventas_ipc["FECHA"].dtype)
print(precios["FECHA"].dtype)

In [ ]:
ventas_ipc = ventas_ipc.merge(precios[["FECHA", "INDICE"]], on="FECHA", how="left")
ventas_ipc.head(3)

In [ ]:
# Creamos una variable que almacene el valor del índice para el 06 de 2022
indice_06_2022 = ventas_ipc[ventas_ipc["FECHA"] == ventas_ipc["FECHA"].max()]["INDICE"].values[0]

La series que están expresadas en pesos son las siguientes: 
   * `Ventas`
   * `Recaudacion`

In [ ]:
ventas_ipc["Ventas_Reales"] = (ventas_ipc["Ventas"]  * indice_06_2022 / ventas_ipc["INDICE"])
ventas_ipc["Recaudacion_Real"] = (ventas_ipc["Recaudacion"]  * indice_06_2022 / ventas_ipc["INDICE"])
ventas_ipc.head(3)

In [ ]:
ventas_agrupadas = ventas_ipc[['FECHA', 'Ventas', 'Ventas_Reales', 'Recaudacion', 'Recaudacion_Real']].copy()
ventas_agrupadas = ventas_agrupadas.groupby('FECHA').sum()[['Ventas', 'Ventas_Reales', 'Recaudacion', 'Recaudacion_Real']].reset_index()

Veamos gráficamente el efecto de la operación (por simplicidad se grafican mensualmente las series de tiempo agregando los valores por suma):

In [ ]:
# Crear el layout con dos subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Primer subplot: Ventas totales vs Ventas reales
ax1.plot(ventas_agrupadas['FECHA'], ventas_agrupadas['Ventas'], label='Total Ventas')
ax1.plot(ventas_agrupadas['FECHA'], ventas_agrupadas['Ventas_Reales'], label='Total Ventas Reales')
ax1.set_xlabel('Fecha')
ax1.set_ylabel('Ventas')
ax1.set_title('Ventas totales vs Ventas reales')
ax1.legend()
ax1.tick_params(axis='x', rotation=45)

# Segundo subplot: Comisión Empresa vs Comisión Empresa Reales
ax2.plot(ventas_agrupadas['FECHA'], ventas_agrupadas['Recaudacion'], label='Comisión Empresa')
ax2.plot(ventas_agrupadas['FECHA'], ventas_agrupadas['Recaudacion_Real'], label='Comisión Empresa Reales')
ax2.set_xlabel('Fecha')
ax2.set_ylabel('Comisión')
ax2.set_title('Comisión Empresa vs Comisión Empresa Reales')
ax2.legend()
ax2.tick_params(axis='x', rotation=45)

# Ajustar el espaciado entre subplots
plt.tight_layout()

# Mostrar el gráfico
plt.show()

En las variantes llamadas "Reales", se observa una pendiente más moderada. El efecto de la inflación ha sido "descartado". De aquí en adelante, a la hora de hablar de los valores de estas variables, la interpretación correcta será "$ XX.XXX a precios de junio de 2022"

In [ ]:
ventas_ipc.columns

In [ ]:
# # Se descartan las variables que no se usan
ventas_ipc = limpiar_basic(ventas_ipc, cols_drop=['Ventas', 'Recaudacion'])

# Se renombran las variables
ventas_ipc.rename(columns = {'FECHA': 'Fecha', 'INDICE': 'IPC', 
                              'Ventas_Reales': 'Ventas', 
                              'Recaudacion_Real': 'Recaudacion'}, inplace = True)

# Se reacomodan las columnas
ventas_ipc = ventas_ipc[['ID', 'Omega', 'Subrubro', 'Trat_Fisc_Agg', 
                          'Trat_Fisc', 'Trat_Dif', 'CM', 'Año', 'Mes', 
                          'Fecha', 'IPC','Deposito', 'Ventas', 
                          'Alicuota', 'Recaudacion', 'Modelo']]

ventas_ipc = ventas_ipc.sort_values(['Año', 'Mes', 'Ventas']).reset_index(drop=True)